In [ ]:
"""
problem tanimi: rakam goruntuleri olusturma
cozum: gan modeli insa edelim
sonuc: generator rakam goruntusu olusturacak
veriseti: mnist
"""

In [76]:
# %% veri seti iceriye aktarma ve preprocessing: normalizasyon, boyutlarin ayarlanmasi
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LeakyReLU # relu varyasyonudur. girdi +x ise cikti +x olur, eger girdi -x ise cikti -ax olur, a kucuk bir sayidir
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, BatchNormalization
from tensorflow.keras.optimizers import Adam #for dongusu sayaci

from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

#mnist veri setini yukle
(X_train,_),(_,_) = mnist.load_data()

#normalization
X_train = X_train / 255.0

# boyutlarin ayarlanmasi (28x28) -> (28,28,1)
X_train = np.expand_dims(X_train, axis = -1)

In [77]:
# %% create discriminator ve generator
def build_discriminator():
    model = Sequential()

    # feature extraction
    # 64 filtre (3x3)
    model.add(Conv2D(64, kernel_size = 3, strides = 2, padding = "same", input_shape = (28,28,1)))
    model.add(LeakyReLU(alpha = 0.2))

    #classification: fake mi? real mi?
    model.add(Flatten()) #Goruntuyu tek boyutlu vektore donusturur
    model.add(Dense(1, activation = "sigmoid")) # output plyer

    # compile
    model.compile(loss = "binary_crossentropy", optimizer = Adam(0.0002,0.5), metrics = ["accuracy"])    
    return model
    
build_discriminator().summary()

zdim = 100
# generator modeli tanimla
def build_generator():
    model = Sequential()
    model.add(Dense(7*7*128, input_dim = zdim)) # gurultu vektorunden yuksek boyutlu uzaya donusum
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Reshape((7,7,128)))
    model.add(BatchNormalization())
    model.add(Conv2DTranspose(64, kernel_size = 3, strides = 2, padding = "same"))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization())
    model.add(Conv2DTranspose(1, kernel_size = 3, strides = 2, padding = "same", activation = "tanh"))

    return model

build_generator().summary()

Model: "sequential_45"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_19 (Conv2D)              │ (None, 14, 14, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_46 (LeakyReLU)      │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_18 (Flatten)            │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 1)              │        12,545 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,185 (51.50 KB)

 Trainable params: 13,185 (51.50 KB)

 Non-trainable params: 0 (0.00 B)

Model: "sequential_46"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_35 (Dense)                │ (None, 6272)           │       633,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_47 (LeakyReLU)      │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_14 (Reshape)            │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_27          │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_26             │ (None, 14, 14, 64)     │        73,792 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_48 (LeakyReLU)      │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_28          │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_27             │ (None, 28, 28, 1)      │           577 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 708,609 (2.70 MB)

 Trainable params: 708,225 (2.70 MB)

 Non-trainable params: 384 (1.50 KB)

In [82]:
# %% create and train gan model = discriminator + generator

# gan modeli olustur
def build_gan(generator, discriminator):
    discriminator.trainable = False # discriminator gan icerisinde egitilmez
    model = Sequential()
    model.add(generator) # gan yapisina ilk olarak generator ekliyoruz
    model.add(discriminator) # discriminator ekleme
    model.compile(loss = "binary_crossentropy", optimizer = Adam(0.0002, 0.5))
    return model

discriminator = build_discriminator()
generator = build_generator()
gan = build_gan(generator, discriminator)
gan.summary()

# gan training 
epochs = 50
batch_size = 64
half_batch = 64//2

# egitim dongusu
for epoch in tqdm(range(epochs), desc= "Training Process"): #tdqm ilerleme cubugu

    # fake veriler ve gercek veriler ile discriminator egitimi
    # gercek veriler ile discriminator egitimi
    idx = np.random.randint(0, X_train.shape[0], half_batch) #x_train icerisinde n rastgele 32 adet half_batch sayisi kadar veri sectik
    real_images = X_train[idx] # gercek goruntuler
    real_labels = np.ones((half_batch, 1)) #gercek goruntu etiketi = 1

    # fake verileri (generatorun urettigi) kullanarak discriminator egitimi
    noise = np.random.normal(0, 1, (half_batch, zdim)) # gurultu vektoru
    #fake_images = generator.predict(noise, verbose = 0) # generator ile uretilen goruntu
    fake_images = generator(noise, training=False)
    fake_labels = np.zeros((half_batch, 1)) # sahte goruntu etiketleri = 0

    # update discriminator
    d_loss_real = discriminator.train_on_batch(real_images, real_labels) # gercek veriler ile loss hesabi
    d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels) # fake veriler ile kayip
    d_loss = np.add(d_loss_real, d_loss_fake) * 0.5 # ortalama kayip

    # gan egitimi
    noise = np.random.normal(0,1, (batch_size, zdim)) #gurultu olustur
    valid_y = np.ones((batch_size, 1)) #dogru etiketler
    g_loss = gan.train_on_batch(noise, valid_y) #gan'in icinde bulunan generatorun egitimi

    if epoch % 100 == 0:
        print(f"{epoch}/{epochs} d_loss: {d_loss[0]}, g_loss: {g_loss}")


Model: "sequential_52"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_51 (Sequential)      │ (None, 28, 28, 1)      │       708,609 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_50 (Sequential)      │ (None, 1)              │        13,185 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 721,794 (2.75 MB)

 Trainable params: 708,225 (2.70 MB)

 Non-trainable params: 13,569 (53.00 KB)

Training Process:   6%|▌         | 3/50 [00:04<00:55,  1.19s/it]

0/50 d_loss: 0.6853572130203247, g_loss: 0.6793252229690552


Training Process: 100%|██████████| 50/50 [00:08<00:00,  5.59it/s]


In [ ]:
# %% test and evaluation: gorsel ve metrik